<a href="https://colab.research.google.com/github/LuciAkirami/nlp-cookbook/blob/main/NLP_3_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] datasets huggingface_hub umap-learn > /dev/null

In this Notebook we will go through the Translation Tasks

Here we are working with the XTREME dataset, to be more precise the PAN dataset which is a subset of XTREME dataset, which contains wikipedia articles in many languages

- whenever we are dealing with a dataset containing multiple domains we can utilize the get_dataset_config_names() to find out which subsets are available

In [2]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")

XTREME has 183 configurations


In [5]:
# we are interested with PAN subset
pan_data = [x for x in xtreme_subsets if x.startswith("PAN")]
pan_data[:5]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de']